Обучаем на всех признаках Age,Sex,RBC,PCV,MCV,MCH,MCHC,RDW,TLC,PLT,HGB

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
from torch.utils.data import TensorDataset, DataLoader

data = pd.read_csv('/content/df11.csv')

data['AgeRisk'] = data['Age'].apply(lambda x: 0.8 if x < 40 else (1.0 if x < 45 else (1.5 if x < 50 else (2.0 if x < 55 else (2.5 if x < 60 else (3.0 if x < 65 else 4.0))))))
data['GenderRisk'] = data['Sex'].apply(lambda x: -0.5 if x == 0 else 0)
data['HGBRisk'] = data['HGB'].apply(lambda x: 3.0 if x < 12 else 0)
data['MCVRisk'] = data['MCV'].apply(lambda x: 0.8 if x < 80 else 0)
data['PLTRisk'] = data['PLT'].apply(lambda x: 0.8 if 290 <= x <= 1000 else 0)

features = ['Age', 'Sex', 'RBC', 'PCV', 'MCV', 'MCH', 'MCHC', 'RDW', 'TLC', 'PLT', 'HGB']
target = ['RiskPercentage']
data['RiskPercentage'] = data['AgeRisk'] + data['GenderRisk'] + data['HGBRisk'] + data['MCVRisk'] + data['PLTRisk']
input_data = data[features].values
target_data = data[target].values

input_tensor = torch.tensor(input_data, dtype=torch.float32)
target_tensor = torch.tensor(target_data, dtype=torch.float32)

batch_size = 32
dataset = TensorDataset(input_tensor, target_tensor)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

class NeuralNet(nn.Module):
    def __init__(self):
        super(NeuralNet, self).__init__()
        self.fc1 = nn.Linear(11, 64)
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, 1)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.fc3(x)
        return x

model = NeuralNet()
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

num_epochs = 15000
best_mae = float('inf')
for epoch in range(num_epochs):
    for inputs, targets in dataloader:
        optimizer.zero_grad()
        output = model(inputs)
        loss = criterion(output, targets)
        loss.backward()
        optimizer.step()

    with torch.no_grad():
        output = model(input_tensor)
        predicted_risk = output.squeeze().detach().numpy()
        mae = torch.mean(torch.abs(output - target_tensor))
        accuracy = torch.mean((torch.abs(output - target_tensor) <= 0.1).float()) * 100

    if (epoch + 1) % 100 == 0:
        print(f'Epoch: {epoch+1}/{num_epochs}, Loss: {loss.item()}, MAE: {mae.item()}, Accuracy: {accuracy.item()}%')

    if mae < best_mae:
        best_mae = mae
        torch.save(model.state_dict(), 'best_model_all_features.pth')

model.load_state_dict(torch.load('best_model_all_features.pth'))

# Прогнозирование на новых данных
new_data = torch.tensor([[28, 0, 5.66, 34, 60.1, 17, 28.2, 20, 11.1, 128.3, 9.6],
                         [41, 0, 4.78, 44.5, 93.1, 28.9, 31, 13, 7.02, 419, 13.8],
                         [40, 1, 4.65, 41.6, 89.5, 28.8, 32.2, 13, 8.09, 325, 13.4]], dtype=torch.float32)
predicted_risk = model(new_data).squeeze().tolist()
print('Predicted Risk Percentages:', predicted_risk)

Epoch: 100/15000, Loss: 0.6522603631019592, MAE: 0.9180006384849548, Accuracy: 6.043956279754639%
Epoch: 200/15000, Loss: 0.9129908084869385, MAE: 0.8633915781974792, Accuracy: 5.494505405426025%
Epoch: 300/15000, Loss: 1.9469175338745117, MAE: 0.7819360494613647, Accuracy: 9.615385055541992%
Epoch: 400/15000, Loss: 0.3334510326385498, MAE: 0.7099802494049072, Accuracy: 9.89011001586914%
Epoch: 500/15000, Loss: 0.3570655286312103, MAE: 0.7034831047058105, Accuracy: 12.912088394165039%
Epoch: 600/15000, Loss: 0.851280689239502, MAE: 0.5812481641769409, Accuracy: 10.439560890197754%
Epoch: 700/15000, Loss: 0.47789284586906433, MAE: 0.5221656560897827, Accuracy: 14.010990142822266%
Epoch: 800/15000, Loss: 0.8681464195251465, MAE: 0.4898674190044403, Accuracy: 16.483516693115234%
Epoch: 900/15000, Loss: 0.6452428102493286, MAE: 0.4667223393917084, Accuracy: 19.78022003173828%
Epoch: 1000/15000, Loss: 0.6796888709068298, MAE: 0.4422973692417145, Accuracy: 16.758241653442383%
Epoch: 1100/150

**Делаем предсказания (11 признаков)**

In [ ]:
# Прогнозирование на новых данных
new_data = torch.tensor([[28, 0, 5.66, 34, 60.1, 17, 28.2, 20, 11.1, 128.3, 9.6],
                         [41, 0, 4.78, 44.5, 93.1, 28.9, 31, 13, 7.02, 419, 13.8],
                         [40, 1, 4.65, 41.6, 89.5, 28.8, 32.2, 13, 8.09, 325, 13.4]], dtype=torch.float32)
predicted_risk = model(new_data).squeeze().tolist()
print('Predicted Risk Percentages:', predicted_risk)

Predicted Risk Percentages: [4.105315208435059, 1.260941982269287, 1.7384669780731201]


**Обучаем на 5 признаках**

**Порядок признаков следующий - 'Age', 'Sex', 'MCV', 'PLT', 'HGB'**

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
from torch.utils.data import TensorDataset, DataLoader

data = pd.read_csv('/content/df5.csv')

data['AgeRisk'] = data['Age'].apply(lambda x: 0.8 if x < 40 else (1.0 if x < 45 else (1.5 if x < 50 else (2.0 if x < 55 else (2.5 if x < 60 else (3.0 if x < 65 else 4.0))))))
data['GenderRisk'] = data['Sex'].apply(lambda x: -0.5 if x == 0 else 0)
data['HGBRisk'] = data['HGB'].apply(lambda x: 3.0 if x < 12 else 0)
data['MCVRisk'] = data['MCV'].apply(lambda x: 0.8 if x < 80 else 0)
data['PLTRisk'] = data['PLT'].apply(lambda x: 0.8 if 290 <= x <= 1000 else 0)

features = ['Age', 'Sex', 'MCV', 'PLT', 'HGB']
target = ['RiskPercentage']
data['RiskPercentage'] = data['AgeRisk'] + data['GenderRisk'] + data['HGBRisk'] + data['MCVRisk'] + data['PLTRisk']
input_data = data[features].values
target_data = data[target].values

input_tensor = torch.tensor(input_data, dtype=torch.float32)
target_tensor = torch.tensor(target_data, dtype=torch.float32)

batch_size = 32
dataset = TensorDataset(input_tensor, target_tensor)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

class NeuralNet(nn.Module):
    def __init__(self):
        super(NeuralNet, self).__init__()
        self.fc1 = nn.Linear(5, 64)
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, 1)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.fc3(x)
        return x

model = NeuralNet()
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

num_epochs = 20000
best_mae = float('inf')
for epoch in range(num_epochs):
    for inputs, targets in dataloader:
        optimizer.zero_grad()
        output = model(inputs)
        loss = criterion(output, targets)
        loss.backward()
        optimizer.step()

    with torch.no_grad():
        output = model(input_tensor)
        predicted_risk = output.squeeze().detach().numpy()
        mae = torch.mean(torch.abs(output - target_tensor))
        accuracy = torch.mean((torch.abs(output - target_tensor) <= 0.1).float()) * 100

    if (epoch + 1) % 100 == 0:
        print(f'Epoch: {epoch+1}/{num_epochs}, Loss: {loss.item()}, MAE: {mae.item()}, Accuracy: {accuracy.item()}%')

    if mae < best_mae:
        best_mae = mae
        torch.save(model.state_dict(), 'best_model.pth')

model.load_state_dict(torch.load('best_model.pth'))

# Прогнозирование на новых данных
new_data = torch.tensor([[28, 0, 60.1, 128.3, 9.6],
                         [41, 0, 93.1, 419, 13.8],
                         [40, 1, 89.5, 325, 13.4]], dtype=torch.float32)
predicted_risk = model(new_data).squeeze().tolist()
print('Predicted Risk Percentages:', predicted_risk)

Epoch: 100/20000, Loss: 1.3933697938919067, MAE: 0.9042531847953796, Accuracy: 7.967033386230469%
Epoch: 200/20000, Loss: 1.9781184196472168, MAE: 0.853081226348877, Accuracy: 6.043956279754639%
Epoch: 300/20000, Loss: 1.4034355878829956, MAE: 0.8039218187332153, Accuracy: 7.967033386230469%
Epoch: 400/20000, Loss: 1.4488903284072876, MAE: 0.7724757790565491, Accuracy: 9.89011001586914%
Epoch: 500/20000, Loss: 1.190330147743225, MAE: 0.7546979188919067, Accuracy: 9.065934181213379%
Epoch: 600/20000, Loss: 1.4181097745895386, MAE: 0.7089589834213257, Accuracy: 7.967033386230469%
Epoch: 700/20000, Loss: 0.31517431139945984, MAE: 0.7112457752227783, Accuracy: 10.164834976196289%
Epoch: 800/20000, Loss: 1.1032131910324097, MAE: 0.674842357635498, Accuracy: 12.087912559509277%
Epoch: 900/20000, Loss: 0.3123861253261566, MAE: 0.6316348314285278, Accuracy: 10.164834976196289%
Epoch: 1000/20000, Loss: 0.617204487323761, MAE: 0.6517340540885925, Accuracy: 14.285715103149414%
Epoch: 1100/20000, 

**Делаем предсказания (5 признаков)**

In [ ]:
new_data = torch.tensor([[28, 0, 60.1, 128.3, 9.6],
                         [41, 0, 93.1, 419, 13.8],
                         [40, 1, 89.5, 325, 13.4]], dtype=torch.float32)
predicted_risk = model(new_data).squeeze().tolist()
print('Predicted Risk Percentages:', predicted_risk)

Predicted Risk Percentages: [4.103352069854736, 1.2889834642410278, 1.8149105310440063]
